In [1]:
import numpy as np
import tensorflow as tf
import time

import utils # from *this directory* -- some helper functions

In [16]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 100
n_train = 60000
n_test = 10000

n_pixels = 28

In [3]:
# Step 1: Read in data
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

Successfully downloaded data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded data/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for train_data

Notes:

    * `tf.data.Dataset.from_tensor_slices()` - create tensors from data in memory
    * `.batch` - Combines consecutive elements of this dataset into batches
    * `tf.data.Iterator`provides the main way to extract elements from a dataset. The operation returned by Iterator.get_next() yields the next element of a Dataset when executed, and typically acts as the interface between input pipeline code and your model.
    * 

In [8]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

In [9]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(img, w) + b 

In [11]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
# note the original function without `_v2` is deprecated so we should transition
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch

In [12]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [13]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [17]:
# Run Model!

with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init) # drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))

Average loss epoch 0: 1.2955552476783132
Average loss epoch 1: 0.7335595782413039
Average loss epoch 2: 0.6005433002183603
Average loss epoch 3: 0.5365793732016586
Average loss epoch 4: 0.49751888735349786
Average loss epoch 5: 0.47063960301321606
Average loss epoch 6: 0.450879934776661
Average loss epoch 7: 0.43526025379812994
Average loss epoch 8: 0.4227807507958523
Average loss epoch 9: 0.4124777170807816
Average loss epoch 10: 0.4036482449880866
Average loss epoch 11: 0.39603594687095905
Average loss epoch 12: 0.38956607947515887
Average loss epoch 13: 0.3836816811630892
Average loss epoch 14: 0.37859724929859473
Average loss epoch 15: 0.3738260543623636
Average loss epoch 16: 0.36946305383083433
Average loss epoch 17: 0.36565559991570407
Average loss epoch 18: 0.3620877629449201
Average loss epoch 19: 0.3589348493966945
Average loss epoch 20: 0.35585100387417995
Average loss epoch 21: 0.3529418185353279
Average loss epoch 22: 0.3503537838888723
Average loss epoch 23: 0.34788326894